# Introduction to Content Safety


Azure Content Safety APIs are used with API key. A secured way for developers to get these keys is from Azure Key Vault. This pattern eliminates the need to store the secrets in your code.

#### Follow [README](https://github.com/tirtho/open-ai/blob/main/README.md) and perform setup before running the notebooks

Reference : 
- [Azure Open AI](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/overview)
- [Azure Content Safety API](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/contentsafety/azure-ai-contentsafety)

#### Load the API key and relevant Python libaries.

In [8]:
#%pip install azure-core
#%pip install azure-ai-contentsafety
#%pip install azure_content_safety_setup

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement azure-core-credentials (from versions: none)
ERROR: No matching distribution found for azure-core-credentials


In [ ]:
! pip show azure-ai-contentsafety
! echo -----------------
! pip show azure-core

In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.contentsafety import ContentSafetyClient
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory
#from azure_content_safety_setup import get_content_safety

In [12]:
# Set environment variables
aics_key = os.environ["CONTENT_SAFETY_KEY"]
aics_endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]
aics_api_version = "2023-10-01"
credential = AzureKeyCredential(aics_key)
client = ContentSafetyClient(aics_endpoint, credential)

#### Analyze text <span style="color:red">without blocklists</span>
Notice the Content Safety system does not detect anything

In [13]:
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory

request = AnalyzeTextOptions(text="I will kil'ya all, you mo**ns.", 
                             categories=[TextCategory.HATE, 
                                         TextCategory.SELF_HARM, 
                                         TextCategory.VIOLENCE,
                                         TextCategory.SEXUAL]
                            )
try:
    response = client.analyze_text(request)
except HttpResponseError as e:
    print("Analyze text failed.")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise

text = 'Content Safety Results:'
# Check the severity (0 to 6)
if response.hate_result:
    text = text + "\n\tHate severity: {}".format(response.hate_result.severity)
if response.self_harm_result:
    text = text + "\n\tSelfHarm severity: {}".format(response.self_harm_result.severity)
if response.violence_result:
    text = text + "\n\tViolence severity: {}".format(response.violence_result.severity)
if response.sexual_result:
    text = text + "\n\tSexual severity: {}".format(response.sexual_result.severity)

print(text)

AttributeError: 'AnalyzeTextResult' object has no attribute 'hate_result'

#### Read lines from a file and check for content safety

In [6]:
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory
import os

from azure_content_safety_setup import get_content_safety

text_file = 'safe_content.txt'
text_path = os.path.join(os.getcwd(),'./data/', text_file)

# Read file line by line and check for content safety
i = 0
with open(text_path) as f:
    for line in f:
        print("[%s]:[%s]:%s" %(text_file, i, line))
        # calling my function to do the job
        result = get_content_safety(client=client, text_input=line)
        print(result)
        i = i + 1

[safe_content.txt]:[0]:You are awesome.

Content Safety Results:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 0
	Sexual severity: 0
[safe_content.txt]:[1]:The world is great.

Content Safety Results:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 0
	Sexual severity: 0


#### Add a block list

In [7]:
from azure_content_safety_setup import add_or_update_block_list

MY_BLOCK_LIST = 'TR_Custom_Blocked_Text_List'
MY_BLOCK_LIST_DESCRIPTION = 'TRs fun list of blocked text strings for testing and demo of the Azure Content Safety APIs'
added_block_list = add_or_update_block_list(client = client, 
                                            block_list_name = MY_BLOCK_LIST, 
                                            block_list_description = MY_BLOCK_LIST_DESCRIPTION)
print("Created Block list name: %s" %(added_block_list))


Blocklist created or updated: 
Created Block list name: TR_Custom_Blocked_Text_List


#### Add blocked text strings

In [8]:
from azure_content_safety_setup import add_blocked_items

my_blocked_text_array = ["kil'ya", "f**king", "m0**ns"]
added_text_list = add_blocked_items(
                        client = client, 
                        block_list_name = MY_BLOCK_LIST, 
                        block_text_array = my_blocked_text_array
                    )
print("Blocked Text items: %s" %(added_text_list))


Add block items failed: 
Error code: InvalidRequestBody
Error message: Some items already exist in list TR_Custom_Blocked_Text_List. items: [m0**ns,kil'ya,f**king]. | Request Id: b3080edf-ff8e-4c11-9d7b-54954f4291cd, Timestamp: 2023-06-15T12:58:19Z.
(InvalidRequestBody) Some items already exist in list TR_Custom_Blocked_Text_List. items: [m0**ns,kil'ya,f**king]. | Request Id: b3080edf-ff8e-4c11-9d7b-54954f4291cd, Timestamp: 2023-06-15T12:58:19Z.
Code: InvalidRequestBody
Message: Some items already exist in list TR_Custom_Blocked_Text_List. items: [m0**ns,kil'ya,f**king]. | Request Id: b3080edf-ff8e-4c11-9d7b-54954f4291cd, Timestamp: 2023-06-15T12:58:19Z.
Blocked Text items: FAILED


#### Analyze text <span style="color:red">with block list</span>
Notice now it is detecting inappropriate content.

In [10]:
from azure_content_safety_setup import get_content_safety_custom

result = get_content_safety_custom(client = client, block_list_names = [MY_BLOCK_LIST], text_input = "I will kil'ya")
print(result)

Content Safety Results (blank if none found):
  Blocklist match results: 
	Block item hit in text, Offset=7, Length=6
	BlocklistName: TR_Custom_Blocked_Text_List, BlockItemId: 1f58ef20-0968-4de6-9231-37d5cedf532e
	BlockItemText: kil'ya


#### Analyze image

In [11]:
from azure_content_safety_setup import get_image_safety
import os

image_file = 'image.jpg'
image_path = os.path.join(os.getcwd(),'./data/', image_file)

result = get_image_safety(client = client, image_filepath = image_path)
print(result)

Image Content Safety Results in file C:\Users\tibarar\OneDrive - Microsoft\Desktop\MyProjects\open-ai\ResponsibleAI\./data/image.jpg:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 2
	Sexual severity: 0
